Machine config:

**CUDA 12.4, GPU: AD104 12GB VRAM, CPU: 12/24 4.4**

### Baseline FP16

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset

device = "cuda:0" if torch.cuda.is_available() else "cpu"
# torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
torch_dtype = torch.float16

In [ ]:
torch.cuda.empty_cache()

In [ ]:
model_id = "openai/whisper-large-v3-turbo"

In [ ]:
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps=True
)

In [ ]:
def splitter(n, s):
    pieces = s.split()
    return (" ".join(pieces[i:i+n]) for i in range(0, len(pieces), n))

In [ ]:
dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]

In [ ]:
result = pipe(sample)
print('\n\n')
for n in splitter(15, result["text"]):
    print(n)

----

### Quantization Int8

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModelForSpeechSeq2Seq

In [ ]:
def quantize_tensor(tensor: torch.Tensor):
    """
    Квантизация float16 tensor в int8
    Возвращает тензор int8 и его scale factor.
    """
    tensor = tensor.float()
    max_abs = tensor.abs().max()
    if max_abs == 0:
        scale = torch.tensor(1.0)
    else:
        scale = max_abs / 127.0
    tensor_int8 = torch.clamp((tensor / scale).round(), -128, 127).to(torch.int8)
    return tensor_int8, scale

In [ ]:
def quantize_model_weights(model: nn.Module):
    """
    Квантизация всех float16 параметров модели.
    Возвращает словарь квантизованных параметров и соответствующий словарь scale factor'ов.
    """
    quantized_state_dict = {}
    scales = {}
    
    for name, param in model.state_dict().items():
        if param.dtype in [torch.float16, torch.float32]:
            q_param, scale = quantize_tensor(param)
            quantized_state_dict[name] = q_param
            scales[name] = scale
        else:
            quantized_state_dict[name] = param
    
    return quantized_state_dict, scales

In [ ]:
model_id = "openai/whisper-large-v3-turbo"
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, low_cpu_mem_usage=True, use_safetensors=True
)
model.eval()

In [ ]:
quantized_state_dict, weight_scales = quantize_model_weights(model)

In [ ]:
checkpoint = {"state_dict": quantized_state_dict, "scales": weight_scales}
torch.save(checkpoint, "whisper_large_v3_turbo_int8.pth")

---